In [ ]:
# %%
# filename: optim_supertrend_vectorbt.ipynb

# --- 1. Imports and Setup ---
import os
import pandas as pd
import numpy as np
import pandas_ta as ta
import vectorbt as vbt

print("Libraries imported.")

In [ ]:
# %%
# --- 2. Configuration ---
# This is the control panel for the optimization.
# Change the parameters here to analyze different datasets or strategy settings.

# --- Data Input ---
# Assumes the file is in the project's 'data/parquet/' directory
DATA_FILENAME = "EURUSD_M15_2024-09-14_to_2025-09-14.parquet"
# The frequency of the data is crucial for correct performance calculation.
# Examples: '15min', '1H', '4H', 'D'
FREQ = '15min'

# --- Optimization Parameters for Supertrend ---
# We define the ranges for the grid search here.
supert_period_range = np.arange(7, 22, 2)    # Test periods from 7 to 21
supert_multiplier_range = np.arange(2.0, 5.1, 0.5) # Test multipliers from 2.0 to 5.0

# --- Backtest Settings ---
INIT_CASH = 10000
FEES = 0.0001      # 0.01% fee per trade
SLIPPAGE = 0.0001  # 0.01% slippage per trade

# --- Analysis Settings ---
# The metric to sort the results by
OPTIMIZE_FOR = 'Sharpe Ratio'
# How many top results to display
N_TOP_RESULTS = 10

# --- Path Construction ---
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
file_path = os.path.join(project_root, "data", "parquet", DATA_FILENAME)

print("Configuration loaded.")

In [ ]:
# %%
# --- 3. Data Loading and Indicator Calculation ---
print(f"Loading data from: {file_path}")
try:
    price_data = pd.read_parquet(file_path)
    price_data.set_index('Time', inplace=True)
    print(f"Data loaded successfully: {len(price_data)} bars.")
except Exception as e:
    print(f"Error loading data: {e}")
    exit()

# --- CORRECTED APPROACH: Create a custom wrapper for pandas-ta ---
# This wrapper function will call pandas-ta and return only the main supertrend line.
# This solves the problem of dynamically named output columns.
def pta_supertrend_wrapper(high, low, close, length, multiplier):
    # Calculate the supertrend using pandas-ta
    st = ta.supertrend(high=high, low=low, close=close, length=length, multiplier=multiplier)

    # pandas-ta returns a DataFrame. The first column is the main supertrend line.
    # We return only this column (as a Series).
    return st.iloc[:, 0]


# Now, create an indicator factory from our OWN wrapper function
Supertrend = vbt.IndicatorFactory(
    input_names=['high', 'low', 'close'],
    param_names=['length', 'multiplier'],
    output_names=['supertrend'] # We can name the output whatever we want here
).from_apply_func(
    pta_supertrend_wrapper,
    # Important settings for functions that expect pandas Series:
    keep_pd=True,
    to_2d=False
)

print("\nCalculating Supertrend for all parameter combinations...")
supertrend_indicator = Supertrend.run(
    price_data['High'],
    price_data['Low'],
    price_data['Close'],
    length=supert_period_range,
    multiplier=supert_multiplier_range,
    param_product=True
)
print("Indicator calculation complete.")

In [ ]:
# %%
# --- 4. Signal Generation and Backtest Execution ---
print("\nGenerating signals and running backtest...")

# Now we can access the output via the name we defined: .supertrend
entries = price_data['Close'].vbt.crossed_above(supertrend_indicator.supertrend)
exits = price_data['Close'].vbt.crossed_below(supertrend_indicator.supertrend)

# Run the portfolio simulation
portfolio = vbt.Portfolio.from_signals(
    price_data['Close'],
    entries,
    exits,
    freq=FREQ,
    init_cash=INIT_CASH,
    fees=FEES,
    slippage=SLIPPAGE
)
print("Backtest complete.")

In [ ]:
# %%
# --- 5. Numerical Analysis of Results (FINAL, DEFINITIVE VERSION 4) ---
print(f"\n--- Optimization Results ---")

if portfolio.trades.count().sum() == 0:
    print("No trades were executed for any parameter combination.")
else:
    # --- Top N Results ---
    print(f"\nTop {N_TOP_RESULTS} parameter combinations sorted by '{OPTIMIZE_FOR}':")

    # Build the results DataFrame manually from individual metric Series.

    total_return = portfolio.total_return()
    sharpe_ratio = portfolio.sharpe_ratio()
    max_drawdown = portfolio.max_drawdown()
    win_rate = portfolio.trades.win_rate()

    # CORRECTED: profit_factor is also an attribute of the `trades` accessor
    profit_factor = portfolio.trades.profit_factor()

    total_trades = portfolio.trades.count()

    # Combine these Series into a single DataFrame.
    results_df = pd.concat([
        total_return,
        sharpe_ratio,
        max_drawdown,
        win_rate,
        profit_factor,
        total_trades
    ], axis=1, keys=[
        'Total Return [%]',
        'Sharpe Ratio',
        'Max Drawdown [%]',
        'Win Rate [%]',
        'Profit Factor',
        'Total Trades'
    ])

    # Sort this new DataFrame by the desired metric.
    top_stats = results_df.sort_values(by=OPTIMIZE_FOR, ascending=False).head(N_TOP_RESULTS)

    print(top_stats)

    # --- Best Result In-Depth ---
    best_params_idx = top_stats.index[0]
    best_portfolio_run = portfolio[best_params_idx]
    best_stats = best_portfolio_run.stats()

    print(f"\n--- Detailed Stats for the Best Combination: {best_params_idx} ---")
    print(best_stats)